<a href="https://colab.research.google.com/github/candyledger1/Express-Certificate-Pricing/blob/main/DL_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment III

# Practical Deep Learning for Language Processing

Author: Nino Maisuradze

Master’s Program Economics and Finance

University of Tübingen

## Fine-Tuning and Preference Alignment of Large Language Models for Airbnb Title Optimization

## 1. Introduction

In digital marketplaces such as Airbnb, listing titles play a crucial role in attracting user attention and influencing click-through behavior. Since potential guests initially only see the listing title, it must clearly communicate key attributes such as location, property type, and standout features.

The goal of this assignment is to develop an automated title generation system based on a pre-trained Large Language Model (LLM). The model is conditioned on Airbnb listing descriptions and trained to generate concise, attractive, and relevant titles.

The assignment consists of the following stages:

	•	Task A: Zero-shot title generation using a base LLM
	•	Task B: Supervised fine-tuning (LoRA + 4-bit quantization)
	•	Task C: Manual human preference labeling
	•	Task D: Preference fine-tuning using Direct Preference Optimization (DPO)
	•	Task E: Comparative evaluation and qualitative analysis
	•	Task F: Deployment via a Gradio web application

The objective is to compare three approaches:
	1.	Zero-shot baseline
	2.	Supervised fine-tuned model (SFT)
	3.	Preference-aligned model (DPO)

and evaluate how each step improves title quality.

## Task A - Zero-Shot Title Geneation

### 2. Zero-Shot Title Generation

In this task, I evaluate how well a pretrained instruction-tuned Large Language Model (LLM) can generate Airbnb listing titles without any additional fine-tuning.

The model is used in a zero-shot setting, meaning it relies solely on its pretrained knowledge and instruction-following capabilities.

The goal is to:

	•	Generate titles conditioned on listing descriptions
	•	Experiment with different prompt formulations
	•	Assess the quality of the generated titles

This zero-shot evaluation serves as a baseline for later comparison with:
	•	Supervised Fine-Tuning (SFT)
	•	Direct Preference Optimization (DPO)

### 2.1 Environment Setup

I first verify that a GPU is available and mount Google Drive to access the dataset.

In [ ]:
!nvidia-smi

Fri Feb 13 15:41:01 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2.2 Experimental Setup

I evaluate the base model (Mistral-7B-Instruct-v0.2) in a zero-shot setting, meaning no additional fine-tuning is applied. The model is used directly as a pretrained instruction-following LLM.

To examine the impact of prompt design, I experiment with three different prompt formulations:

	•	v1: Simple instruction
	•	v2: More structured format
	•	v3: Stronger stylistic constraints

For each prompt version, I generate titles for 8 Airbnb listings and compare the outputs qualitatively.



### 2.3 Load Dataset

In [ ]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/airbnb_tabular.csv"

df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()

(22570, 116)


,Unnamed: 0,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,...,has_amenity_Elevator,has_amenity_Host greets you,has_amenity_Free parking on premises,len_amenities,len_description,proxy,review_diff,in_top_third,img_available,joint_description
0,0.0,13913,https://www.airbnb.com/rooms/13913,2.022061e+13,2022-06-08,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730.0,...,0,0,1,41,154,3,15.0,1,1.0,My bright double bedroom with a large window h...
1,3.0,17402,https://www.airbnb.com/rooms/17402,2.022061e+13,2022-06-08,Superb 3-Bed/2 Bath & Wifi: Trendy W1,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",https://a0.muscache.com/pictures/39d5309d-fba7...,67564.0,...,1,0,0,38,112,3,5.0,1,1.0,You'll have a wonderful stay in this superb mo...
2,4.0,25123,https://www.airbnb.com/rooms/25123,2.022061e+13,2022-06-08,Clean big Room in London (Room 1),Big room with double bed clean sheets clean to...,Barnet is one of the largest boroughs in Londo...,https://a0.muscache.com/pictures/456905/a004b9...,103583.0,...,0,0,0,14,129,1,0.0,0,1.0,Big room with double bed clean sheets clean to...
3,5.0,36299,https://www.airbnb.com/rooms/36299,2.022061e+13,2022-06-07,Kew Gardens 3BR house in cul-de-sac,3 Bed House with garden close to Thames river ...,"Residential family neighborhood, with both Eng...",https://a0.muscache.com/pictures/457052/6e819d...,155938.0,...,0,0,0,34,128,3,7.0,1,1.0,3 Bed House with garden close to Thames river ...
4,9.0,39387,https://www.airbnb.com/rooms/39387,2.022061e+13,2022-06-08,Stylish bedsit in Notting Hill ish flat.,Private lockable bedsit room available within ...,My place is convenient for all London attracti...,https://a0.muscache.com/pictures/beda1dab-9443...,168920.0,...,0,1,0,40,135,1,0.0,0,1.0,Private lockable bedsit room available within ...


In [ ]:
df_top = df[df["in_top_third"] == 1].copy()

print(df_top.shape)
df_top[["description", "name"]].head()

(5782, 116)


,description,name
0,My bright double bedroom with a large window h...,Holiday London DB Room Let-on going
1,You'll have a wonderful stay in this superb mo...,Superb 3-Bed/2 Bath & Wifi: Trendy W1
3,3 Bed House with garden close to Thames river ...,Kew Gardens 3BR house in cul-de-sac
6,A luminous room in a modern 2 bedroom flat loc...,Room with a view zone 1 Central Bankside
8,Blenheim Lodge was built in 1878 when there we...,You Will Save Money Here


### 2.4 Load Instruction-Tuned Base Model

I now load the pretrained model used in the assignment:

	•	Mistral-7B-Instruct-v0.2
	•	4-bit quantized (to fit Colab GPU memory)

In [ ]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import login

# Read secret from Colab Secrets panel
hf_token = userdata.get("HF_TOKEN")

# Login to HuggingFace
login(hf_token)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

# Important for Mistral padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model.eval()

print("Model loaded successfully.")

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Model loaded successfully.


### 2.5 Prompt Construction

To guide the model, I design instruction-style prompts that condition title generation on the listing description.

Rather than using a single prompt, I experiment with three different prompt formulations to examine how prompt design influences zero-shot performance.

Each prompt takes the listing description as input and instructs the model to generate a short Airbnb-style title.

In [ ]:
def build_prompt_v1(description):
    # Simple direct instruction
    return f"""Write a short and attractive Airbnb listing title.

Description:
{description}

Title:"""

def build_prompt_v2(description):
    # Role-based instruction
    return f"""You are an expert Airbnb copywriter.
Create a short, catchy, and professional listing title
based on the description below.

Description:
{description}

Title:"""

def build_prompt_v3(description):
    # More constrained / structured
    return f"""Generate a concise Airbnb listing title (max 10 words).
Focus on location and key selling points.

Description:
{description}

Title:"""

### 2.6 Inference Function

In [ ]:
def generate_title(description, prompt_builder, max_new_tokens=20):
    prompt = prompt_builder(description)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only text after "Title:"
    if "Title:" in generated_text:
        return generated_text.split("Title:")[-1].strip()
    else:
        return generated_text.strip()

### 2.7 Example Zero-Shot Generation

In [ ]:
N = 10   # start small for testing
subset = df_top.iloc[:N].copy()

titles_v1 = []
titles_v2 = []
titles_v3 = []

for desc in subset["description"]:
    titles_v1.append(generate_title(desc, build_prompt_v1))
    titles_v2.append(generate_title(desc, build_prompt_v2))
    titles_v3.append(generate_title(desc, build_prompt_v3))

subset["title_v1"] = titles_v1
subset["title_v2"] = titles_v2
subset["title_v3"] = titles_v3

subset[["name", "title_v1", "title_v2", "title_v3"]]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

,name,title_v1,title_v2,title_v3
0,Holiday London DB Room Let-on going,"Cozy Double Room in Finsbury Park, Central Lon...",Bright & Cozy Double Room in Central Finsbury ...,Bright Double Room in Finsbury Park: Comfortab...
1,Superb 3-Bed/2 Bath & Wifi: Trendy W1,Modern & Spacious 3-Bedroom Apartment in Centr...,"Modern Fitzrovia Apartment: Beautiful 3-Bed, 2...","Modern, Spacious 3-Bed 2-Bath Apartment in Cen..."
3,Kew Gardens 3BR house in cul-de-sac,Charming 3 Bed House by the Thames with easy a...,"""River Retreat: Charming 3-Bed House with Gard...","Modern 3-bed house near Thames, Kew Gardens & ..."
6,Room with a view zone 1 Central Bankside,"Bright, Modern & Quiet Room with Stunning Shar...","""Bright & Boutique: Luxe Veggie Flat - King Ro...","Bright, King-sized Room with Shard View in Cen..."
8,You Will Save Money Here,Historic 1878 Blenheim Lodge - Quiet Upmarket ...,"""Blenheim Lodge: Charming 1878 Victorian Home ...","Historic East Finchley House - Double Room, Gr..."
11,Beautiful 1 bed apt in Queens Park,Chic Mid-Century Design Apartment with Balcony...,"""Mid-Century Chic: Bright, View-Filled Flat w/...",Modern Mid-Century London Apartment with Balco...
12,Quiet Comfortable Room in Fulham,"Charming Double Single Room in Quiet, Safe Nei...",Charming Double Single in Quiet Munster Villag...,"Quiet, Safe Double Room in Munster Village - S..."
13,"Beautiful, Luxurious Art Deco +private bathroom",Luxe Art Deco Flat: Unique Private Room w/Ensu...,"""Luxe Retreat in a Charming Art Deco Flat - Ro...","Trendy Art Deco Flat: Ensuite Room with View, ..."
15,Cosy Double studio in Zone 2 Hammersmith (6),"""Cozy & Convenient Studio in Hammersmith - Min...","Hip Hammersmith Hub: Minutes from Kensington, ...",Modern Studios in Hammersmith - Convenient Bas...
18,Cosy Double studio in Zone 2 Hammersmith (1),"Modern, Bright and Convenient Studio Apartment...",Hammersmith Haven - Conveniently Located Studi...,Modern Hammersmith Studio - Close to Kensingto...


In [ ]:
# Save results for comparison

SAVE_PATH = "/content/drive/MyDrive/zero_shot_results.csv"

subset_to_save = subset[["name", "title_v1", "title_v2", "title_v3"]].copy()

subset_to_save.to_csv(SAVE_PATH, index=False)

print("Zero-shot results saved to:", SAVE_PATH)
subset_to_save.head()

Zero-shot results saved to: /content/drive/MyDrive/zero_shot_results.csv


,name,title_v1,title_v2,title_v3
0,Holiday London DB Room Let-on going,"Cozy Double Room in Finsbury Park, Central Lon...",Bright & Cozy Double Room in Central Finsbury ...,Bright Double Room in Finsbury Park: Comfortab...
1,Superb 3-Bed/2 Bath & Wifi: Trendy W1,Modern & Spacious 3-Bedroom Apartment in Centr...,"Modern Fitzrovia Apartment: Beautiful 3-Bed, 2...","Modern, Spacious 3-Bed 2-Bath Apartment in Cen..."
3,Kew Gardens 3BR house in cul-de-sac,Charming 3 Bed House by the Thames with easy a...,"""River Retreat: Charming 3-Bed House with Gard...","Modern 3-bed house near Thames, Kew Gardens & ..."
6,Room with a view zone 1 Central Bankside,"Bright, Modern & Quiet Room with Stunning Shar...","""Bright & Boutique: Luxe Veggie Flat - King Ro...","Bright, King-sized Room with Shard View in Cen..."
8,You Will Save Money Here,Historic 1878 Blenheim Lodge - Quiet Upmarket ...,"""Blenheim Lodge: Charming 1878 Victorian Home ...","Historic East Finchley House - Double Room, Gr..."


### 2.8 Results

In this task, I evaluated the instruction-tuned base model (Mistral-7B-Instruct-v0.2) in a zero-shot setting without any additional fine-tuning.

I generated Airbnb listing titles for 8 descriptions using three different prompt formulations. This allowed me to examine how prompt design influences the quality of generated titles.

Overall, I observe that:

	•	The simple prompt (v1) produces generally relevant titles but sometimes lacks stylistic consistency or strong selling points.
	•	The more structured prompt (v2) leads to clearer and more professional titles.
	•	The constrained prompt (v3) often generates the most concise and Airbnb-style outputs, focusing on key features and location.

The experiment shows that even without fine-tuning, the base instruction-tuned LLM is capable of generating coherent and attractive listing titles. However, the quality and consistency depend strongly on prompt formulation.

This zero-shot evaluation serves as a baseline for comparison with supervised fine-tuning (SFT) and preference optimization (DPO) in later tasks.

# Task B: Supervised Fine-Tuning (SFT)

### 3. Supervised Fine-Tuning with LoRA and 4-bit Quantization

In this task, I fine-tune the base instruction-tuned model to generate Airbnb titles from listing descriptions using supervised fine-tuning (SFT).

To make training feasible on limited GPU memory, I apply:

	•	4-bit quantization (QLoRA setup)
	•	LoRA (Low-Rank Adaptation) for parameter-efficient training

Only a small set of additional trainable parameters is updated, while the base model remains frozen.

After fine-tuning, I generate titles for the same 5–10 descriptions from Task A using a consistent prompt format and compare performance to the zero-shot baseline.

In [ ]:
# Step 1 - install dependencies

!pip install -q transformers accelerate bitsandbytes peft trl datasets

### 3.2 import Libraries


In [ ]:
import torch
import pandas as pd
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

DATA_PATH = "/content/drive/MyDrive/airbnb_tabular.csv"
df = pd.read_csv(DATA_PATH)

# keep only needed columns and drop missing
df_train = df[["description", "name"]].dropna().copy()

# keep small for stability (increase later)
df_train = df_train.sample(n=1000, random_state=42).reset_index(drop=True)

def format_sample(row):
    return (
        "You are an expert Airbnb copywriter.\n\n"
        f"Description:\n{row['description']}\n\n"
        f"Title:\n{row['name']}"
    )

df_train["text"] = df_train.apply(format_sample, axis=1)
dataset = Dataset.from_pandas(
    df_train[["text"]],
    preserve_index=False
)

### 3.3 Load Base Model in 4-bit (QLoRA Setup)

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,  # <- critical for T4
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

# recommended for training stability
model.config.use_cache = False

# prepare for k-bit training (QLoRA setup)
model = prepare_model_for_kbit_training(model)

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

### 3.4 Apply LoRA

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # keep light
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


### 3.5 Supervised Fine-Tuning (SFT)

In this section, I fine-tune the instruction-tuned base model (Mistral-7B-Instruct-v0.2) to improve title generation performance.

To make fine-tuning feasible on limited GPU memory, I apply:

	•	4-bit quantization (BitsAndBytes) to reduce memory footprint
	•	LoRA (Low-Rank Adaptation) to train only a small number of additional parameters

This allows efficient parameter-efficient fine-tuning without updating the full model.

In [ ]:
MAX_LEN = 256

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./mistral_lora_airbnb",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="no",
    fp16=True,          # ✅ correct for T4
    bf16=False,         # ❗ must be False
    report_to="none",
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.219845
20,2.068080
30,2.072100
40,1.971469
50,2.034188
60,1.973187
70,2.074577
80,1.952936
90,1.953888
100,1.986139


TrainOutput(global_step=125, training_loss=2.0221002349853516, metrics={'train_runtime': 1897.4376, 'train_samples_per_second': 0.527, 'train_steps_per_second': 0.066, 'total_flos': 8445995410391040.0, 'train_loss': 2.0221002349853516, 'epoch': 1.0})


The training loss decreased steadily, indicating that the LoRA adapters successfully adapted the model to the Airbnb title generation task.

Next, I evaluate the fine-tuned model on the same subset used in the zero-shot experiment.

In [ ]:
model.save_pretrained("/content/drive/MyDrive/mistral_lora_run1")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral_lora_run1")

('/content/drive/MyDrive/mistral_lora_run1/tokenizer_config.json',
 '/content/drive/MyDrive/mistral_lora_run1/chat_template.jinja',
 '/content/drive/MyDrive/mistral_lora_run1/tokenizer.json')

## Task B.2 Train Second Model

In [ ]:
import torch
import pandas as pd
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

In [ ]:
DATA_PATH = "/content/drive/MyDrive/airbnb_tabular.csv"
df = pd.read_csv(DATA_PATH)

df_train_2 = df[["description", "name"]].dropna().copy()
df_train_2 = df_train_2.sample(n=1000, random_state=42).reset_index(drop=True)

def format_sample_2(row):
    return (
        "You are an expert Airbnb copywriter.\n\n"
        f"Description:\n{row['description']}\n\n"
        f"Title:\n{row['name']}"
    )

df_train_2["text"] = df_train_2.apply(format_sample_2, axis=1)

dataset_2 = Dataset.from_pandas(
    df_train_2[["text"]],
    preserve_index=False
)

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config_2 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# LOAD TOKENIZER FIRST
tokenizer_2 = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer_2.pad_token is None:
    tokenizer_2.pad_token = tokenizer_2.eos_token

# THEN tokenize
MAX_LEN = 256

def tokenize_function_2(example):
    return tokenizer_2(
        example["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )

tokenized_dataset_2 = dataset_2.map(
    tokenize_function_2,
    batched=True,
    remove_columns=["text"],
)

# THEN load model
model_2 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config_2,
    device_map="auto",
)

model_2 = prepare_model_for_kbit_training(model_2)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

In [ ]:
MAX_LEN = 256

def tokenize_function_2(example):
    return tokenizer_2(
        example["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )

tokenized_dataset_2 = dataset_2.map(
    tokenize_function_2,
    batched=True,
    remove_columns=["text"],
)

tokenizer_2 = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer_2.pad_token is None:
    tokenizer_2.pad_token = tokenizer_2.eos_token

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### LoRa Config

In [ ]:
lora_config_2 = LoraConfig(
    r=32,  # higher rank
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model_2 = get_peft_model(model_2, lora_config_2)
model_2.print_trainable_parameters()

trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879


### Training Config

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args_2 = TrainingArguments(
    output_dir="./mistral_lora_airbnb_run2",
    per_device_train_batch_size=2,   # changed
    gradient_accumulation_steps=4,   # changed
    num_train_epochs=2,              # changed
    learning_rate=1e-4,              # changed
    logging_steps=50,
    save_strategy="no",
    fp16=True,
    bf16=False,
    report_to="none",
)

In [ ]:
data_collator_2 = DataCollatorForLanguageModeling(
    tokenizer=tokenizer_2,
    mlm=False,
)

In [ ]:
trainer_2 = Trainer(
    model=model_2,
    args=training_args_2,
    train_dataset=tokenized_dataset_2,
    data_collator=data_collator_2,
)

In [ ]:
trainer_2.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,2.078186
100,1.988707
150,1.920784
200,1.925752
250,1.904663


TrainOutput(global_step=250, training_loss=1.963618408203125, metrics={'train_runtime': 2703.7012, 'train_samples_per_second': 0.74, 'train_steps_per_second': 0.092, 'total_flos': 1.979136751185101e+16, 'train_loss': 1.963618408203125, 'epoch': 2.0})

In [ ]:
SAVE_PATH_2 = "/content/drive/MyDrive/mistral_lora_airbnb_run2"

model_2.save_pretrained(SAVE_PATH_2)
tokenizer_2.save_pretrained(SAVE_PATH_2)

print("Model 2 saved to:", SAVE_PATH_2)

Model 2 saved to: /content/drive/MyDrive/mistral_lora_airbnb_run2


# Task B.3 Evaluation

I have completed three approaches for title generation: (i) a zero-shot baseline using the instruction-tuned base model (Mistral-7B-Instruct-v0.2) with three prompt variants, and (ii) two LoRA fine-tuned models trained on Airbnb-style title generation data. I saved the zero-shot outputs as a CSV and saved both fine-tuned adapters to Google Drive.

Now, I load the saved zero-shot results and reload both LoRA adapters on top of the same base model. Then I generate titles for the same evaluation subset with Model 1 and Model 2, store their outputs next to the zero-shot outputs, and save a final comparison CSV for qualitative evaluation (readability, relevance, specificity, and style consistency).

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load Models
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

device = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

# ---------- ZERO SHOT MODEL ----------
tokenizer_zero = AutoTokenizer.from_pretrained(BASE_MODEL)
model_zero = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

# ---------- FINETUNED MODEL 1 ----------
PATH_MODEL_1 = "/content/drive/MyDrive/mistral_lora_run1"

tokenizer_1 = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model_1 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

model_1 = PeftModel.from_pretrained(base_model_1, PATH_MODEL_1)

# ---------- FINETUNED MODEL 2 ----------
PATH_MODEL_2 = "/content/drive/MyDrive/mistral_lora_airbnb_run2"

tokenizer_2 = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model_2 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

model_2 = PeftModel.from_pretrained(base_model_2, PATH_MODEL_2)

print("All models loaded successfully.")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

All models loaded successfully.


In [ ]:
# Unified Generation Function
def generate_title(model, tokenizer, description):
    prompt = (
        "You are an expert Airbnb copywriter.\n\n"
        f"Description:\n{description}\n\n"
        "Title:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            temperature=0.8,
            top_p=0.9,
            do_sample=True
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text.split("Title:")[-1].strip()

In [ ]:
# load zero shot dataset
data = pd.read_csv("/content/drive/MyDrive/airbnb_tabular.csv")
subset = data[["description", "name"]].dropna().head(10)

In [ ]:
# Generate Comparison
results = []

for idx, row in subset.iterrows():
    desc = row["description"]

    title_zero = generate_title(model_zero, tokenizer_zero, desc)
    title_1 = generate_title(model_1, tokenizer_1, desc)
    title_2 = generate_title(model_2, tokenizer_2, desc)

    results.append({
        "original": row["name"],
        "zero_shot": title_zero,
        "finetuned_1": title_1,
        "finetuned_2": title_2
    })

results_df = pd.DataFrame(results)
results_df.head(10)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

,original,zero_shot,finetuned_1,finetuned_2
0,Holiday London DB Room Let-on going,Relaxed Double Room with Queen Size Bed in Cen...,"Double Bedroom in Finsbury Park, London (Zone ...",Large double room in Finsbury Park (2 blocks a...
1,Superb 3-Bed/2 Bath & Wifi: Trendy W1,"Modern, Spacious 3-Bedroom Apartment in Centra...","Wonderful recently refurbished apartment, cent...",Beautiful Contemporary 3-Bed 2-Bath Apartment ...
2,Clean big Room in London (Room 1),Spacious Double Room with Garden Access in Qui...,"Clean, Quiet, Bright, Double Room for 6 Months...","Bright, spacious room, in clean house 7 mins t..."
3,Kew Gardens 3BR house in cul-de-sac,"""Serene River Retreat: 3-Bed House in Quiet Cu...","3 bed House close to Thames river, Central Lon...",Family home near Thames river and Kew Gardens ...
4,Stylish bedsit in Notting Hill ish flat.,Bright & Convenient Bedsit Room - Quiet Street...,Private double bedroom in quiet location. Brig...,Bedroom in private flat - 10 mins to tube stat...


In [ ]:
SAVE_EVAL_PATH = "/content/drive/MyDrive/evaluation_results.csv"
results_df.to_csv(SAVE_EVAL_PATH, index=False)

print("Saved to:", SAVE_EVAL_PATH)

Saved to: /content/drive/MyDrive/evaluation_results.csv


In [ ]:
results_df.head(10)

,original,zero_shot,finetuned_1,finetuned_2
0,Holiday London DB Room Let-on going,Relaxed Double Room with Queen Size Bed in Cen...,"Double Bedroom in Finsbury Park, London (Zone ...",Large double room in Finsbury Park (2 blocks a...
1,Superb 3-Bed/2 Bath & Wifi: Trendy W1,"Modern, Spacious 3-Bedroom Apartment in Centra...","Wonderful recently refurbished apartment, cent...",Beautiful Contemporary 3-Bed 2-Bath Apartment ...
2,Clean big Room in London (Room 1),Spacious Double Room with Garden Access in Qui...,"Clean, Quiet, Bright, Double Room for 6 Months...","Bright, spacious room, in clean house 7 mins t..."
3,Kew Gardens 3BR house in cul-de-sac,"""Serene River Retreat: 3-Bed House in Quiet Cu...","3 bed House close to Thames river, Central Lon...",Family home near Thames river and Kew Gardens ...
4,Stylish bedsit in Notting Hill ish flat.,Bright & Convenient Bedsit Room - Quiet Street...,Private double bedroom in quiet location. Brig...,Bedroom in private flat - 10 mins to tube stat...
5,Clean big Room in London (Room 2),"Large, Bright, and Cozy Double Room in Quiet a...",Nice big double room to let in Golders Green 7...,BIG ROOM LET IN BRENT CROSS NORTH LONDON 1 MON...
6,Room with a view zone 1 Central Bankside,"Bright, Calm and Quiet Central London Room wit...",Luminous room in Central London with great vie...,Modern and bright bedroom in central London wi...
7,Room in maisonette in chiswick,Cozy Double Room with Balcony in Chiswick - Pe...,"Bedroom with own bathroom, zone 2, 100m from h...","Double room in nice 1 bed flat in Chiswick, Lo..."
8,You Will Save Money Here,Experience History in a Charming Victorian Hom...,Double room in North London - zone 3. Good val...,Blenheim Lodge - lovely guest room in London h...
9,2 Double bed apartment in quiet area North London,Cozy Ground Floor Apartment with Private Parki...,Stylish apartment with private garden and park...,Stylish 2 Bedroom Flat With Parking In Mill Hi...


## Task B: Evaluation and Analysis

I compare three approaches:
- Zero-shot baseline
- Supervised Fine-Tuned Model 1 (SFT)
- Supervised Fine-Tuned Model 2 (SFT)

### Overall Observations

**Zero-shot model**
- Often creative and descriptive.
- Sometimes too long and slightly promotional.
- Occasionally includes unnecessary phrases (e.g., “Subtitle:”).
- Captures amenities well but lacks consistency in style.

**Fine-Tuned Model 1**
- More structured and closer to Airbnb-style titles.
- Includes relevant details like location and number of bedrooms.
- Sometimes contains formatting artifacts (e.g., “Link:”, “Description:”).
- Slightly verbose in some cases.

**Fine-Tuned Model 2**
- More concise and cleaner.
- Titles feel more natural and aligned with Airbnb style.
- Better balance between informativeness and readability.
- Fewer formatting errors compared to Model 1.

### Comparison Summary

Across the examples, the fine-tuned models generally produce titles that are more aligned with Airbnb conventions than the zero-shot baseline.

Model 2 appears to generate the most consistent and concise titles, while Model 1 provides strong factual coverage but sometimes includes noise. The zero-shot model is creative but less controlled.

Overall, supervised fine-tuning improves relevance, clarity, and stylistic alignment compared to the base model.

# Task C: Human Preference Labeling